# Рекомендация тарифов

Оператор мобильной связи выяснил: многие клиенты пользуются архивными тарифами. Он хочет построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф.

Наша задача — построить модель со значением **accuracy** не менее 0,75.

Основными этапами нашего проекта станут:  
* Изучение данных
* Разделение данных на выборки
* Исследование качества разных моделей
* Проверка качества модели на тестовой выборке
* Проверка модели на вменяемость

**Описание данных**  

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:  
**сalls** — количество звонков,  
**minutes** — суммарная длительность звонков в минутах,  
**messages** — количество sms-сообщений,  
**mb_used** — израсходованный интернет-трафик в Мб,  
**is_ultra** — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).


Проект выполнен в **Jupyter Notebook**, версия сервера блокнотов: 6.1.4. Версия **Python** 3.7.8.  
В проекте использованы библиотеки: 
* **Pandas**
* **NumPy**
* **scikit-learn**
* **IPython**

## Изучение данных

In [1]:
# Импортируем необходимые библиотеки и модули.
import pandas as pd
import numpy as np
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
# Изучим датасет.
data = pd.read_csv('/datasets/users_behavior.csv')
display(data)
data.info()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [2]:
# Необходимости хранить данные в столбцах calls и messages 
# в формате float64 нет.
# Переведем их в формат int64.
data = data.astype({'calls':'int', 'messages':'int'})
display(data)
data.info()

,calls,minutes,messages,mb_used,is_ultra
0,40,311.90,83,19915.42,0
1,85,516.75,56,22696.96,0
2,77,467.66,86,21060.45,0
3,106,745.53,81,8437.39,1
4,66,418.74,1,14502.75,0
...,...,...,...,...,...
3209,122,910.98,20,35124.90,1
3210,25,190.36,0,3275.61,0
3211,97,634.44,70,13974.06,0
3212,64,462.32,90,31239.78,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 125.7 KB


### Вывод

Мы располагаем данными о поведении клиентов, которые уже перешли на тарифы Смарт и Ультра.  
Это таблица с 5 столбцами и 3214 строками.  
Форматы столбцов:  
* *calls*, *minutes*, *messages*, *mb_used* — `float64`
* *is_ultra* — `int64`
  
Данные уже предобработаны нами в предыдущем проекте. Пропусков нет. Однако проведен перевод формата столбцов *calls* и *messages* из `float64` в `int64`, так как нет смысла содержать количество звонков и sms-сообщений в формате с плавающей запятой.

## Разделение данных на выборки

In [3]:
# Подготовим данные для последующей разбивки на выборки.
# Столбец is_ultra - целевой признак.
# Извлекаем признаки.
features = data.drop('is_ultra', axis=1)
# Извлекаем целевой признак.
target = data['is_ultra']
# Разобьем данные на тренировочный, валидационный и тестовый наборы.
# Сначала поделим на тренировочный и валидационный. 
# Размер валидационной выборки 0.2.
features_train, features_valid, target_train, target_valid = train_test_split(
    features,
    target, 
    test_size=.2,
    random_state=12345
)
# Теперь разделим тренировочную выборку на тестовую и 
# окончательный вариант тренировочной выборки.
features_train, features_test, target_train, target_test = train_test_split(
    features_train,
    target_train, 
    test_size=.25,
    random_state=12345
)
# Выборки готовы. Взглянем на их размеры. 
# Мы помним, что изначальный набор данных 
# имел 5 признаков и 3214 объектов.
samples = [
    features_train, target_train, 
    features_valid, target_valid, 
    features_test, target_test
]
for sample in samples:
    display(sample.shape)

(1928, 4)

(1928,)

(643, 4)

(643,)

(643, 4)

(643,)

### Вывод

Мы определили целевой и остальные признаки. Целевым признаком являются данные из столбца *is_ultra*. Данные этого столбца сохранены в переменной `target`. Остальные столбцы — остальные признаки. Они сохранены в переменной `features`.

При помощи метода `train_test_split` мы разделили исходные данные на три выборки: тренировочную (60 % от исходного набора), валидационную (20 % от исходного набора) и тестовую (20 % от исходного набора).  
  Данные сохранены в соответствующих переменных: `features_train`, `target_train`, `features_valid`, `target_valid`, `features_test`, `target_test`.  
Перед нами задача классификации. Исследуем три модели: дерево решений, случайный лес, логистическую регрессию.

## Исследование качества разных моделей

In [4]:
# Ислледуем модель дерева решений, проверив значение accuracy 
# с разными максимальными глубинами дерева.
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    print('max_depth =', depth, ': ', end='')
    print(accuracy_score(target_valid, predictions_valid))

max_depth = 1 : 0.7480559875583204
max_depth = 2 : 0.7838258164852255
max_depth = 3 : 0.7869362363919129
max_depth = 4 : 0.7869362363919129
max_depth = 5 : 0.7884914463452566


In [5]:
# Исследуем модель случайный лес, проверив значение accuracy 
# с разным количеством оценщиков.
best_model = None
best_result = 0
for est in range(1, 11):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model = model
        best_result = result
print(est)
print("Accuracy наилучшей модели на валидационной выборке:", best_result)

10
Accuracy наилучшей модели на валидационной выборке: 0.7869362363919129


In [6]:
# Теперь исследуем логистическую регрессию.
model = LogisticRegression(solver='lbfgs', random_state=12345)
model.fit(features_train, target_train)
result = model.score(features_valid, target_valid)
print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.7589424572317263


### Вывод

Нами было исследовано качество таких моделей как дерево решений, случайный лес, логистическая регрессия.  
При исследовании качества моделей дерева решений и случайного леса, мы меняли такие гиперпараметры, как максимальная глубина и количество оценщиков.  
Лучшего всего себя показали дерево решений с максимальной глубиной дерева 5 и случайный лес с количеством оценщиков 10.  
Принято решение проверить на тестовой выборке модель случайного леса, так как эта модель обеспечивает более высокое качество. Невысокой скоростью можно пренебречь, так как мы располагаем небольшим датасетом.

## Проверка модели на тестовой выборке

In [7]:
# Проверим выбранную модель на тестовой выборке.
model = RandomForestClassifier(random_state=12345, n_estimators=10)
model.fit(features_train, target_train)
result = model.score(features_test, target_test)
print("Accuracy наилучшей модели на тестовой выборке:", result)

Accuracy наилучшей модели на тестовой выборке: 0.7884914463452566


### Вывод

Проверка модели на тестовой выборке показала качество 0.7884914463452566. Задача выполнена, так как качество превышает заданный уровень 0,75. Тем не менее, проверим модель на адекватность (sanity check).

## Проверка модели на адекватность

In [8]:
# Создадим простую модель, делающую предсказания. 
# Сравним её значение accuracy со значением нашей модели.
dummy = DummyClassifier(random_state=12345)
dummy.fit(features_train, target_train)
dummy.score(features_test, target_test)

0.6889580093312597

### Вывод

При помощи `DummyClassifier` нами была создана простая модель, делающая предсказания. Мы обучили её на тренировочном наборе и протестировали на тестовом. Значение **accuracy** составило 0.6889580093312597, что меньше значения нашей модели.  
Можно сделать вывод, что наша модель адекватна. Качество 0.7884914463452566 можно признать довольно высоким.

## Общий вывод

Мы изучили предоставленный нам набор данных. Определили признаки, разбили их на целевые и остальные. Разбили набор на три выборки: тренировочную, валидационную и тестовую. Меняя гиперпараметры моделей, мы проверяли их качество на валидационных выборках. В результате мы приняли решение тестировать модель случайного леса. Уровень качества предсказаний нашей модели на тестовой выборке составил 0.7884914463452566. Сравнив его с качеством dummy-модели мы убедились в высоком качестве нашей модели.

In [9]:
conclusion = pd.DataFrame(
    index=['Результаты'],
    columns=['Целевой признак', 
              'Доля тренировочной выборки', 
              'Доля валидационной выборки', 
              'Доля тестовой выборки', 
              'Выбранная модель', 
              'Гиперпараметр модели', 
              'Качество на валидационной выборке', 
              'Качество на тестовой выборке', 
              'Качество Dummy-модели', 
              'Адекватность модели (Да/Нет)'],
    data=[['is_ultra', 
           '60 %',
           '20 %',
           '20 %', 
           'Случайный лес',
           'n_estimators=10',
           0.7869,
           0.7885, 
           0.6890, 
           'Да']]
)
conclusionStyler = conclusion.style.set_properties(
    **{'text-align': 'center'}
)
conclusionStyler.set_table_styles(
    [dict(selector='th', props=[('text-align', 'center')])]
)
display(conclusionStyler)

,Целевой признак,Доля тренировочной выборки,Доля валидационной выборки,Доля тестовой выборки,Выбранная модель,Гиперпараметр модели,Качество на валидационной выборке,Качество на тестовой выборке,Качество Dummy-модели,Адекватность модели (Да/Нет)
Результаты,is_ultra,60 %,20 %,20 %,Случайный лес,n_estimators=10,0.786900,0.788500,0.689000,Да
